In [1]:


from network import R2Plus1DClassifier
import torch
lip_model = R2Plus1DClassifier(num_classes=11, layer_sizes=[2,2,2,2,2])
state_dicts = torch.load("test_model_puremodel", map_location = torch.device("cuda:0"))
lip_model.load_state_dict(state_dicts)
lip_model.eval()



RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 1024.00 MiB total capacity; 538.38 MiB already allocated; 8.11 MiB free; 542.00 MiB reserved in total by PyTorch)

In [3]:
buffer  = torch.zeros(1,3,10,50,100).cuda()
lip_model.cuda()
%time lip_model(buffer)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 1024.00 MiB total capacity; 540.39 MiB already allocated; 6.11 MiB free; 544.00 MiB reserved in total by PyTorch)

In [3]:
import torch.onnx

In [4]:
torch.onnx.export(lip_model,               # model being run
                  buffer,                         # model input (or a tuple for multiple inputs)
                  "res2_5size50_100.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {2 : 'time_step'}})

In [3]:
import onnxruntime
print(onnxruntime.get_device())

CPU


In [2]:
import psutil
import onnxruntime
import numpy as np

so = onnxruntime.SessionOptions()
so.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
so.optimized_model_filepath = "optimized.onnx"
so.intra_op_num_threads=psutil.cpu_count(logical=True)

ort_session = onnxruntime.InferenceSession("k5l2_2_30_60_ball.onnx", so)

buffer = np.zeros((1,3,200,50,100),dtype = "float32")

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: buffer}

# %time torch_out = lip_model(buffer)

# # compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

RuntimeError: D:\7\s\onnxruntime\core\providers\cuda\cuda_call.cc:123 onnxruntime::CudaCall D:\7\s\onnxruntime\core\providers\cuda\cuda_call.cc:117 onnxruntime::CudaCall CUDA failure 999: unknown error ; GPU=960 ; hostname=DESKTOP-SIBMUSQ ; expr=cudaSetDevice(device_id_); 



In [21]:
buffer = np.random.rand(1,3,150,30,60).astype('float32')
ort_inputs = {ort_session.get_inputs()[0].name:buffer }


In [22]:
%time ort_outs = ort_session.run(None, ort_inputs)

Wall time: 700 ms


In [34]:
ort_session.get_inputs()[0].name: to_numpy(buffer)}

SyntaxError: unmatched '}' (<ipython-input-34-974912626c73>, line 1)

In [5]:
ort_inputs

{'input': array([[[[[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          ...,
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0.,

In [33]:
inp.dtype

dtype('float64')

In [44]:
npiunput =  np.zeros((1,3,30,30,60),dtype = 'float32')

In [45]:
npiunput.expand_dims( axis=0)

AttributeError: 'numpy.ndarray' object has no attribute 'expand_dims'